In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, scatter, etc.)
using CairoMakie

import NamedTrajectories as NT

- convert_arguments is not good because it cannot plot multiple lines

In [ ]:
function convert_arguments(
    P::MakieCore.PointBased, 
    traj::NT.NamedTrajectory{<:Any},
    component::Int,
)
    times = NT.get_times(traj)
    positions = [(times[j], traj.data[component, j]) for j in eachindex(times)]
    return convert_arguments(P, positions)
end

# define the default plot type (preserve `plot` calls)
MakieCore.plottype(::NT.NamedTrajectory, ::Int) = Lines


# NOTE: The rest of these conversions should be recipes!
# -----

# function convert_arguments(
#     P::MakieCore.PointBased, 
#     traj::NT.NamedTrajectory{<:Any},
#     name::Symbol,
#     index::Int
# )
#     times = NT.get_times(traj)
#     positions = [(times[j], traj[name][index, j]) for j in eachindex(times)]
#     return convert_arguments(P, positions)
# end

# function convert_arguments(
#     P::MakieCore.PointBased, 
#     traj::NT.NamedTrajectory{<:Any},
#     name::Symbol
# )
#     times = NT.get_times(traj)
#     rows, cols = size(traj[name])
#     positions = vec([Point2(times[j], traj[name][i, j]) for i in 1:rows, j in 1:cols])
#     return convert_arguments(P, positions)
#     # return vec(positions)
# end


In [ ]:
traj = rand(NT.NamedTrajectory, 10);

In [ ]:
f = Figure()
ax = Axis(f[1, 1])
[lines!(ax, traj, i) for i in traj.components[:u]]
f

Notes
- Makie defaults to Makie.Axis as the preferred axis.

In [ ]:
@recipe(MyPlot, traj, name) do scene
    Theme(
        Lines = (
            linewidth = 10,
            linestyle = :dash,
        ),
        Axis = (
            leftspinevisible = false,
        ),
    )
    # Attributes(
    #     labels = false,
    # )
end

Makie.preferred_axis_type(P::MyPlot) = Makie.Axis

function Makie.plot!(P::MyPlot{<:Tuple{<:NT.NamedTrajectory, Symbol}})
    lift(P[:traj], P[:name]) do traj, name
        for i in traj.components[name]
            lines!(P, traj, i)
        end
    end
    return P
end

# How to deal with multiple symbols?

In [ ]:
f = Figure()
ax = Axis(f[1, 1])
myplot!(ax, traj, :x)

myplot(f[1, 2], traj, :u)

f